In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
groq_api_key = os.environ.get('GROQ_API_KEY')

In [17]:
from langchain_groq import ChatGroq

chat = ChatGroq(
    model="llama3-groq-8b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=groq_api_key
    # other params...
)
llm = chat

In [7]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = chat.invoke(messages)
ai_msg

AIMessage(content='I love programming in French is: "J\'aime programmer."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 30, 'total_tokens': 46, 'completion_time': 0.023908558, 'prompt_time': 0.002670042, 'queue_time': 0.010758778, 'total_time': 0.0265786}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-7c6999b9-005e-435a-9429-8c4b5ae30fc6-0', usage_metadata={'input_tokens': 30, 'output_tokens': 16, 'total_tokens': 46})

In [10]:
for chunk in chat.stream("Write me a song about goldfish on the moon"):
    print(chunk.content, end="", flush=True)

(Verse 1)
Up in the sky, in a world so new,
There's a goldfish bowl, with a view.
Swimming around, in weightless delight,
In the moon's soft glow, through the endless night.

(Chorus)
Goldfish on the moon, in a silvery sea,
Living their lives, so carefree.
In a place we've only dreamed, they call it home,
Oh, the tales they could tell, if they could only roam.

(Verse 2)
They dance with the stars, and play with the light,
In their shimmering world, everything's just right.
No predators here, no need to hide,
Just a peaceful life, far and wide.

(Chorus)
Goldfish on the moon, in a crater so bright,
Living their dreams, in the lunar night.
In a place we've yet to tread, they've made their own,
Oh, the secrets they keep, in their watery throne.

(Bridge)
In their silver castle, they twirl and they spin,
In a world where no one else has ever been.
A tiny piece of Earth, in the cosmic expanse,
Home to these fish, with their heavenly dance.

(Chorus)
Goldfish on the moon, in a world so pure,

In [11]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

tools = [add, multiply]

In [18]:
llm_with_tools = llm.bind_tools(tools)

In [27]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_r9f4',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_pp8z',
  'type': 'tool_call'}]

In [23]:
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wstx', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_9y1b', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_6vky', 'function': {'arguments': '{"a": 40, "b": 4}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 343, 'total_tokens': 436, 'completion_time': 0.08023527, 'prompt_time': 0.047237227, 'queue_time': 0.001767684999999998, 'total_time': 0.127472497}, 'model_name': 'llama3-groq-8b-8192-tool-use-preview', 'system_fingerprint': 'fp_260dc69250', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7478c361-1885-424a-b642-5c6362e04d8d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_wstx', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_9y1b', 'type': 'tool_call'}, {'name': 'divide'

In [25]:
from pydantic import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

In [28]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]